## 번개장터 데이터 수집을 위한 크롤러

- 번개장터 데이터를 수집하기 위한 크롤러 입니다.
- 크롤링하는 과정을 함수화(모듈화)하여 편의성을 높입니다.
- 크롤링 후에는 중복 제거, 날짜컬럼 생성의 과정을 거칩니다.
- 키워드 리스트를 만들어 for문을 수행합니다.

### import package

In [3]:
import requests # 크롤링을 위함
import json # json파일을 다루기위함
import time # 크롤링에 딜레이를 두기 위함
from datetime import datetime, timedelta # 날짜데이터 변환을 위함

import pandas as pd
import numpy as np

# 데이터프레임이 짤려보이는 것을 해결해줌
pd.options.display.max_rows=100
pd.options.display.max_columns=100

### List for crawling

- 크롤링할 대상이되는 데이터를 리스트로 정리한다.
- keywords 리스트와 url이 있다.
- 이 문서의 마지막에 모든 키워드를 크롤링하는 함수가 있다.


**[사용법]**
1. keywords 리스트에 검색어를 차례로 입력한다.
2. url은 keyword에 해당되는 Request URL을 입력하였고, 이는 키워드가 달라져도 변함이 없다.<br>
    1. url은 개발자도구 Network 탭에서 페이지클릭 후 find_v2.json파일에 해당되는 Request URL을 가져온다.
    2. find_v2.json?q={} => 상품명을 {}로 변환한다 (함수 내에서 포맷팅을 위해)
    3. order=date&page={} => 페이지를 {}로 변환한다 (함수 내에서 포맷팅을 위해)
    4. 검색 시점에 해당되는 request_id가 있으나 시간데이터로 모두 같은 url을 사용함으로써 통일시킨다.

In [8]:
keywords = ['나이키 덩크로우 범고래', '뉴발란스 992 그레이']
url = 'https://api.bunjang.co.kr/api/1/find_v2.json?q={}&order=date&page={}&request_id=20211110194853&stat_uid=77848616&token=c63b703f121940d189c222b3335d80ed&stat_device=w&n=100&stat_category_required=1&req_ref=search&version=4'

### Crawler

- keyword와 url, n을 변수로 받아 크롤링을 진행한다.
- keyword와 url은 리스트이고, n은 1페이지부터 n페이지까지 크롤링해준다. (**default=20** : 20페이지 * 100 = 2000개)
- 데이터프레임을 반환받으며, 크롤러 함수를 변수에 선언해주어야한다.

In [4]:
def bunkae_crawler(keyword, n=20):
    df = pd.DataFrame()
    
    for i in range(0,n):

        # 차단막는 코드, 랜덤으로 time.sleep 지정
        seed = np.random.randint(100)
        np.random.seed(seed)
        a = np.random.randint(3)
        time.sleep(a)
        
        url_formating = url.format(keyword, i) # url에 포맷팅을 적용한다.
        info = {
        'referer': 'https://m.bunjang.co.kr/',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
        } # info는 변화없음

        print(f'{i}th, {(i+1)*100}개째 상품을 크롤링 중')
        # requests로 데이터 요청하기
        resp = requests.get(url_formating, headers = info)

        if resp.status_code == requests.codes.ok:
            data = json.loads(resp.text)
            next_df = pd.DataFrame(data['list'])
            df = pd.concat([df, next_df])

        else:
            print(f'{i}번째 요청이 잘못되었습니다.')
            pass # 넘겨준다.

    print('크롤링 완료!!')
    result = df.reset_index(drop=True)
    return result

In [10]:
# test : '나이키 덩크로우 범고래'를 5페이지만 가져와보자
bunkae = bunkae_crawler(keywords[1], 5)

0th, 100개째 상품을 크롤링 중
1th, 200개째 상품을 크롤링 중
2th, 300개째 상품을 크롤링 중
3th, 400개째 상품을 크롤링 중
4th, 500개째 상품을 크롤링 중
크롤링 완료!!


In [5]:
len(bunkae['pid'].value_counts())

500

### Data copy

- 크롤링을 다시 진행하는 것을 막기 위해 데이터 조작 전, 백업한다.
- 뒤에서 작업은 bunkae_df를 사용한다.

In [6]:
bunkae_df = bunkae.copy()

### Deduplicator

- 크롤링 과정에서 상품이 등록되면 한칸씩 밀리면서 같은 상품이 두번 크롤링되는 경우가 있다. 이를 해결하기 위한 함수이다.
- pid(상품명)를 기준으로 중복을 제거한다. (상품명은 unique id)

In [5]:
def deduplicator(df):
    df = df.drop_duplicates(['pid'], keep='first')
    return df

In [8]:
len(deduplicator(bunkae_df)['pid'])

500

### Datetime maker

- update_time 컬럼 : 1970-01-01 09:00 이후로 초를 카운트한 값이다.<br>
    - 1970-01-01 09:00라는 기준에서 5분 이내의 오차가 존재한다. (큰 문제 없음)
- update_time 컬럼을 datetime과 timedelta를 이용해 날짜로 변환한다.
- 새로 생성된 컬럼<br>
    - datetime : timedelta형식의 날짜데이터
    - year : 년도데이터
    - month : 월 데이터
    - day : 일 데이터
    - weekday : 요일 데이터
    - hour : 시간 데이터

In [6]:
def get_date(x):
    return datetime(1970,1,1,9) + timedelta(seconds=int(x))

In [7]:
def datetime_maker(df):
    df['datetime'] = df['update_time'].apply(get_date)
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['week'] = df['datetime'].dt.week
    df['weekday'] = df['datetime'].dt.weekday
    df['hour'] = df['datetime'].dt.hour
    return df

In [11]:
bunkae_df.head(1)

,ad,bizseller,checkout,contact_hope,free_shipping,is_adult,is_super_up_shop,location,max_cpc,name,num_comment,num_faved,only_neighborhood,outlink_url,pid,price,product_image,pu_id,ref_campaign,ref_code,ref_medium,ref_content,ref_source,status,style,super_up,tag,uid,update_time,used,bun_pay_filter_enabled,imp_id,ad_ref,faved
0,False,False,False,False,True,False,None,서울특별시 송파구 거여1동,None,뉴발란스 993 그레이 d 280 250,0,0,False,,168685991,260000,https://media.bunjang.co.kr/product/168685991_...,None,None,soldout_test_v2:B,None,,,0,,None,뉴발란스 993 스탠다드 280 992,11897777,1636589813,2,False,2993618c61dc0fd4f325,,False


In [12]:
bunkae_df = datetime_maker(bunkae_df)

<ipython-input-10-3d26374500e2>:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week'] = df['datetime'].dt.week


In [13]:
bunkae_df.head(1)

,ad,bizseller,checkout,contact_hope,free_shipping,is_adult,is_super_up_shop,location,max_cpc,name,num_comment,num_faved,only_neighborhood,outlink_url,pid,price,product_image,pu_id,ref_campaign,ref_code,ref_medium,ref_content,ref_source,status,style,super_up,tag,uid,update_time,used,bun_pay_filter_enabled,imp_id,ad_ref,faved,datetime,year,month,day,week,weekday,hour
0,False,False,False,False,True,False,None,서울특별시 송파구 거여1동,None,뉴발란스 993 그레이 d 280 250,0,0,False,,168685991,260000,https://media.bunjang.co.kr/product/168685991_...,None,None,soldout_test_v2:B,None,,,0,,None,뉴발란스 993 스탠다드 280 992,11897777,1636589813,2,False,2993618c61dc0fd4f325,,False,2021-11-11 09:16:53,2021,11,11,45,3,9


# Final Keywords Crawling

- 위의 함수들을 모두 적용한 셀이며, 이것만 실행하면 keywords 모두 다 크롤링해온다.

In [8]:
keywords = ['조던1 로우']
url = 'https://api.bunjang.co.kr/api/1/find_v2.json?q={}&order=date&page={}&request_id=20211110194853&stat_uid=77848616&token=c63b703f121940d189c222b3335d80ed&stat_device=w&n=100&stat_category_required=1&req_ref=search&version=4'

In [10]:
# test로 keyword별로 500개씩만 가져와본다. (n=5 : default는 2000개씩)
bunkae = pd.DataFrame()
for keyword in keywords:
    print(f'{keyword} 크롤링 시작!')
    shoe_df = bunkae_crawler(keyword, n=50) # n == 페이지수
    bunkae = pd.concat([bunkae, shoe_df])
    print('----'*10)
print('모든 신발 크롤링 완료!')
bunkae = bunkae.reset_index(drop=True)
bunkae_df = bunkae.copy()
deduplicator(bunkae_df)
bunkae_df = datetime_maker(bunkae_df)

조던1 로우 크롤링 시작!
0th, 100개째 상품을 크롤링 중
1th, 200개째 상품을 크롤링 중
2th, 300개째 상품을 크롤링 중
3th, 400개째 상품을 크롤링 중
4th, 500개째 상품을 크롤링 중
5th, 600개째 상품을 크롤링 중
6th, 700개째 상품을 크롤링 중
7th, 800개째 상품을 크롤링 중
8th, 900개째 상품을 크롤링 중
9th, 1000개째 상품을 크롤링 중
10th, 1100개째 상품을 크롤링 중
11th, 1200개째 상품을 크롤링 중
12th, 1300개째 상품을 크롤링 중
13th, 1400개째 상품을 크롤링 중
14th, 1500개째 상품을 크롤링 중
15th, 1600개째 상품을 크롤링 중
16th, 1700개째 상품을 크롤링 중
17th, 1800개째 상품을 크롤링 중
18th, 1900개째 상품을 크롤링 중
19th, 2000개째 상품을 크롤링 중
20th, 2100개째 상품을 크롤링 중
21th, 2200개째 상품을 크롤링 중
22th, 2300개째 상품을 크롤링 중
23th, 2400개째 상품을 크롤링 중
24th, 2500개째 상품을 크롤링 중
25th, 2600개째 상품을 크롤링 중
26th, 2700개째 상품을 크롤링 중
27th, 2800개째 상품을 크롤링 중
28th, 2900개째 상품을 크롤링 중
29th, 3000개째 상품을 크롤링 중
30th, 3100개째 상품을 크롤링 중
31th, 3200개째 상품을 크롤링 중
32th, 3300개째 상품을 크롤링 중
33th, 3400개째 상품을 크롤링 중
34th, 3500개째 상품을 크롤링 중
35th, 3600개째 상품을 크롤링 중
36th, 3700개째 상품을 크롤링 중
37th, 3800개째 상품을 크롤링 중
38th, 3900개째 상품을 크롤링 중
39th, 4000개째 상품을 크롤링 중
40th, 4100개째 상품을 크롤링 중
41th, 4200개째 상품을 크롤링 중
42th, 4300개째 상품을 크롤링 중
43th, 4400개째 상품

<ipython-input-7-3d26374500e2>:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week'] = df['datetime'].dt.week


In [11]:
bunkae_df['name']

0                               조던1 로우 그린토 260
1                      트래비스스캇 프라그먼트 조던1 로우 245
2                             조던1로우 스타피쉬 W 245
3                                   조던1로우 파인그린
4                   [250] 에어 조던 11로우 스네이크 라이트본
                         ...                  
4455                          조던 11 브레드 로우 팝니다
4456                           (250)조던11 로우 뱀피
4457    [230] 조던1 로우 se orange blaze 팝니다 - 새제품
4458         오시야/나이키 조던 로우/상태A+/사이즈(240)/S1290
4459                             에어조던 1 브레드 로우
Name: name, Length: 4460, dtype: object

**Raw Data 수집 완료**

### Data Save

In [12]:
bunkae_df.to_csv('./NIKE_Jordan1Low.csv')
# 불러올때는 첫번째컬럼 제거해 주어여함 bunkae_df.drop(bunkae_df.columns[0], axis=1)

In [ ]:
bunkae_df.drop(bunkae_df.columns[0], axis=1)